In [1]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


In [2]:
!pip install peft --quiet

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Remove nrows = ? for 

In [4]:
# df = pd.read_csv('/kaggle/input/cleaned-news-dataset/Cleaned_news_dataset.csv', usecols=['summary', 'article'], nrows=1000)

In [5]:
# df = pd.read_csv('/kaggle/input/cleaned-news-dataset/Cleaned_news_dataset.csv', usecols=['summary', 'article'])

In [6]:
from datasets import load_dataset
import pandas as pd

# Load the Bengali portion of the XL-Sum dataset
dataset = load_dataset('csebuetnlp/xlsum', 'bengali')

# Combine all splits (train, validation, test) into one
combined_data = pd.concat(
    [
        pd.DataFrame(dataset[split])
        for split in dataset.keys()
    ],
    ignore_index=True
)

# Keep only 'text' (article) and 'summary' columns
combined_data = combined_data[['text', 'summary']].rename(columns={'text': 'article'})

# Display the first few rows
print(combined_data.head())
df=combined_data

                                             article  \
0  দুর্গাপুজায় মহিষাসুর বধ্যে মধ্য দিয়ে অশুভর ও...   
1  গত ২০ বছরে তিনি রাশিয়ার প্রেসিডেন্ট এবং প্রধা...   
2  বরিশাল সদরে চলছে নির্বাচনী প্রচার প্রচারণা। যদ...   
3  রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কা...   
4  আর কুমিল্লা থেকে বাস ভাড়া করে অনেকের সাথে নিজ...   

                                             summary  
0  হিন্দু বাঙালীরা যে সময়ে তাদের সবথেকে বড় উৎসব...  
1  ভ্লাদিমির পুতিন তাঁর ক্ষমতায় থাকার ২০ বছর পূর...  
2  বাংলাদেশের দক্ষিণাঞ্চলীয় জেলা বরিশাল এখন তুমু...  
3  দু'হাজার চার সালে বিবিসি বাংলা একটি 'শ্রোতা জর...  
4  'ভাষণ শুরু আগে মাথার উপর দিয়ে বিমান আর হেলিকপ...  


In [7]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)


In [8]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

# Model on Alpaca-orca dataset

In [9]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-bangla-alpaca-orca-instruct-v0.0.1")
model = AutoModelForCausalLM.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-bangla-alpaca-orca-instruct-v0.0.1")

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,   # Task type
    inference_mode=False,           # Enable training mode
    r=16,                           # LoRA rank
    lora_alpha=32,                  # Scaling factor
    lora_dropout=0.1                # Dropout rate
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.to(device)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 2048)
          )
        )
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): Parameter

# Model on culturax dataset

In [10]:
# from peft import get_peft_model, LoraConfig, TaskType
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load the model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-unolp-culturax-base-v0.0.1")
# model = AutoModelForCausalLM.from_pretrained("BanglaLLM/BanglaLLama-3.2-1b-unolp-culturax-base-v0.0.1")

# # Configure LoRA
# lora_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,   # Task type
#     inference_mode=False,           # Enable training mode
#     r=16,                           # LoRA rank
#     lora_alpha=32,                  # Scaling factor
#     lora_dropout=0.1                # Dropout rate
# )

# # Wrap the model with LoRA
# model = get_peft_model(model, lora_config)
# model.to(device)


## Tokenize dataset with your model tokenizer

In [11]:
# Tokenize the data
def preprocess_function(examples):
    inputs = tokenizer(
        examples["article"], max_length=256, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["summary"], max_length=256, truncation=True, padding="max_length"
    )

    # Replace pad token ID with -100 in labels (ignored during loss computation)
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"],
    }


In [12]:
from transformers import DataCollatorForSeq2Seq




tokenized_train = ds_train.map(
    preprocess_function, batched=True, remove_columns=["summary", "article", "__index_level_0__"]
)
tokenized_test = ds_test.map(
    preprocess_function, batched=True, remove_columns=["summary", "article", "__index_level_0__"]
)



# Define a data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    return_tensors="pt"
)


Map:   0%|          | 0/7088 [00:00<?, ? examples/s]

Map:   0%|          | 0/3038 [00:00<?, ? examples/s]

# Debug stuff

In [13]:
print(tokenized_train[0])


{'input_ids': [128000, 11372, 105, 50228, 224, 11372, 110, 50228, 99, 60008, 11372, 114, 60008, 36278, 116, 11372, 105, 11372, 248, 60008, 11372, 107, 11372, 120, 60008, 36278, 250, 87648, 11372, 103, 53906, 108, 81278, 107, 11372, 120, 36278, 116, 50228, 106, 50228, 250, 81278, 243, 36278, 106, 50228, 100, 53906, 107, 11372, 106, 36278, 117, 11372, 110, 36278, 104, 60008, 11372, 116, 11372, 105, 28025, 223, 11372, 243, 100278, 36278, 116, 11372, 106, 53906, 103, 53906, 108, 11372, 97, 62456, 36278, 107, 28025, 223, 11372, 243, 53906, 97, 73358, 50228, 115, 53906, 253, 53906, 108, 60008, 73358, 36278, 101, 81278, 231, 11372, 229, 11372, 107, 11372, 120, 73358, 53906, 243, 60008, 36278, 116, 28025, 224, 73358, 53906, 107, 11372, 106, 28025, 223, 11372, 244, 28025, 222, 36278, 104, 28025, 223, 11372, 110, 60008, 73358, 36278, 237, 11372, 243, 36278, 105, 50228, 245, 50228, 101, 60008, 36278, 103, 73358, 81278, 105, 50228, 108, 36278, 101, 81278, 107, 11372, 120, 60008, 36278, 246, 28025,

In [14]:
train_loader = DataLoader(tokenized_train, batch_size=2, collate_fn=data_collator)
batch = next(iter(train_loader))
print({key: val.shape for key, val in batch.items()})

{'input_ids': torch.Size([2, 256]), 'attention_mask': torch.Size([2, 256]), 'labels': torch.Size([2, 256])}


In [15]:
batch = {k: v.to(device) for k, v in batch.items()}
outputs = model(**batch)
print("Loss:", outputs.loss)
print("Logits shape:", outputs.logits.shape)


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Loss: tensor(13.6622, device='cuda:0', grad_fn=<NllLossBackward0>)
Logits shape: torch.Size([2, 256, 128256])


In [16]:
model.gradient_checkpointing_enable()

batch = next(iter(train_loader))
batch = {k: v.to(device) for k, v in batch.items()}
outputs = model(**batch)
print("Logits:", outputs.logits.shape)

# Compute loss and backpropagate
loss = outputs.loss
loss.backward()
print(torch.cuda.memory_allocated(device) / 1024**3, "GB used after backward pass")


Logits: torch.Size([2, 256, 128256])
7.015652656555176 GB used after backward pass


# ------------------------------------------------------------------

# Train

In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"
print("ran")

ran


In [18]:
# for name, param in model.named_parameters():
#     if not param.requires_grad:
#         print(f"Parameter {name} does not require gradients!")


In [19]:
# for param in model.parameters():
#     param.requires_grad = True


In [20]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",          # Save model here
    eval_strategy="epoch",    # Evaluate every epoch
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=False,     # Generate summaries during eval
    logging_dir='./logs',           # For logging
    logging_steps=100,
    save_strategy="no",
    report_to="none",
    fp16 = True,
    dataloader_num_workers=2,        # Optimize data loading
    gradient_accumulation_steps=4,   # Adjust if batch size is small
    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
1,3.415100,3.405940
2,3.396700,3.394353
3,3.393300,3.391777


TrainOutput(global_step=1329, training_loss=3.8410122069853245, metrics={'train_runtime': 8113.4271, 'train_samples_per_second': 2.621, 'train_steps_per_second': 0.164, 'total_flos': 4.104423734457139e+16, 'train_loss': 3.8410122069853245, 'epoch': 3.0})

## Save model?

In [21]:
# Save the model and tokenizer
trainer.save_model("/kaggle/working/saved_model")  # Save in Kaggle working directory
tokenizer.save_pretrained("/kaggle/working/saved_model")  # Save tokenizer files


('/kaggle/working/saved_model/tokenizer_config.json',
 '/kaggle/working/saved_model/special_tokens_map.json',
 '/kaggle/working/saved_model/tokenizer.json')

In [22]:
import shutil

# Compress the saved model directory
shutil.make_archive("/kaggle/working/saved_model", 'zip', "/kaggle/working/saved_model")


'/kaggle/working/saved_model.zip'

# Get ROUGE scores

In [23]:
pip install rouge_score evaluate --quiet

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
    preds, labels = eval_arg

    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring

    # (Note : Please change this code, when you perform on other languages except for Bengali)
    text_preds = [(p if p.endswith(("!", "!", "?", "?", "।")) else p + "।") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "!", "?", "?", "।")) else l + "।") for l in text_labels]
    sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
    )

## Dataloader takes data in random sequence randomly, so results may vary

In [25]:
from torch.utils.data import DataLoader

testing_dataloader = DataLoader(
    tokenized_test.with_format("torch"),
    collate_fn=data_collator,
    batch_size=1
)

for batch in testing_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=2,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_new_tokens=256
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'rouge1': 0.6708074534161491,
 'rouge2': 0.49896049896049893,
 'rougeL': 0.33126293995859213,
 'rougeLsum': 0.49897750511247446}